In [2]:
# Import the dependencies.
from flask import Flask, jsonify
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect


#################################################
# Database Setup
#################################################

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(autoload_with=engine) #use automap to reflect database

# reflect the tables
measure = Base.classes.measurement
station = Base.classes.station

# Save references to each table


# Create our session (link) from Python to the DB
session = Session(engine)

#################################################
# Flask Setup
#################################################
app = Flask(__name__)



#################################################
# Flask Routes
#################################################
@app.route("/")
def welcome():
    return (
        f"Welcome to the Hawaii weather API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0<br/>"
    )

@app.route("/api/v1.0/precipitation")
def prcp():
    lastdate = session.query(measure.date).order_by(measure.date.desc()).first()
    lastdatedt = dt.datetime.strptime(lastdate[0],'%Y-%m-%d')
    yearago = lastdatedt - dt.timedelta(days=365)
    prcpdata = session.query(measure.prcp,measure.date).filter(measure.date >=yearago).all()
    prcponly =[]
    dateonly=[]
    for i in range(len(prcpdata)):
        prcponly.append(prcpdata[i][0])
        dateonly.append(prcpdata[i][1])
    prcp_dict=dict(zip(dateonly,prcponly))
    return jsonify(prcp_dict)